# Feature Selection

Kaynak: https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2

In [23]:
import pandas as pd
import numpy as np

data = pd.read_csv("breast_cancer.csv") 
X = data.drop(["diagnosis","Unnamed: 32","Id"],axis=1)
y = data["diagnosis"]
y = [1 if i=="B" else 0 for i in y]

In [24]:
print(X.columns)
print("count :",len(X.columns))

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')
count : 30


## Neden Özellik Seçiyoruz ?

* Çok fazla girdi overfitting'e neden oluyor.
* Basit girdili modeller daha açıklanabilirdirler.
* Çöp girerse çöp çıkar. Modelin eğitimi ve çalışma maliyeti ve performansı çok etkilenir.

### Özellik Seçmede Temelde 3 Yöntem var

* **Filter-Based**

Bu yöntemlerde bir metriğe göre özellikler filtrelenir. Örneğin pearson corelasyonuna yada chi-square değerine göre.<br>

* **Wrapper-Based**

Bu temeldeki yöntemler özellik seçimini bir arama problemi olarak ele alırlar.<br>

* **Embeded**

Bazı algoritmaların kendi özellik seçme yöntemleri vardır. Örneğin; Lasso, Random Forest

## Bazı Yöntemler

### 1 - Pearson Correlation

![](img/corelation.png)

Bu yöntem Filter-Based bir yöntemdir. Burada **corelasyon değerinin mutlağına bakılarak belirlenen en yüksek n adet özellik** seçilir.

In [39]:
num_feats = 20

In [40]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

20 selected features


In [41]:
cor_feature

['concave points_se',
 'texture_mean',
 'symmetry_worst',
 'smoothness_worst',
 'texture_worst',
 'area_se',
 'perimeter_se',
 'radius_se',
 'compactness_worst',
 'compactness_mean',
 'concavity_worst',
 'concavity_mean',
 'area_mean',
 'radius_mean',
 'area_worst',
 'perimeter_mean',
 'radius_worst',
 'concave points_mean',
 'perimeter_worst',
 'concave points_worst']

### 2 - Chi-Square

Bu yöntemde filter based bir yöntemdir. **En yüksek chi-squere değerine sahip özellikler** seçilir.

In [42]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

20 selected features


In [43]:
chi_feature

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'radius_se',
 'perimeter_se',
 'area_se',
 'concave points_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst']

### 3 - Recursive Feauture Elimination (RFE)

Bu yöntem wrapper based bir yöntemdir. Belirtildiği gibi özellik seçimini bir arama problemi olarak ele alır.

Özyinelemeli özelliklerin kaldırılmasının (RFE) amacı, daha küçük ve daha küçük özellik kümelerini tekrar tekrar düşünerek özellikleri seçmektir. İlk olarak, tahmin edici ilk özellik kümesi üzerinde eğitilir ve her bir özelliğin önemi bir **coef_** niteliği veya bir **feature_importances_** niteliği aracılığıyla elde edilir. Ardından, en az önemli özellikler mevcut özellikler kümesinden budanır. Bu prosedür, budanan sette tekrar tekrar seçilmek istenen özellik sayısına ulaşılıncaya kadar tekrarlanır.

**Bu yöntemi bir tahmin edici yöntem ile kullanabilirsiniz.**

In [44]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 30 features.
20 selected features


D:\YuklenenProgramlar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\YuklenenProgramlar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
rfe_feature

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'concavity_mean',
 'concave points_mean',
 'fractal_dimension_mean',
 'radius_se',
 'perimeter_se',
 'area_se',
 'compactness_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst']

### 4 - Lasso ve Tree Based : SelectFromModel

Bu yöntemler embeded denilen yöntemlerdir. Bu yöntemler zaten algoritmanın içinde bulunan yöntemlerdir. Bu algoritmalra örnek olarak Lasoo ve Random Forest verilebilir. Örneğin Lassoda Lasso Regularizer bir çok özelliğin ağırlığını  0 yapmaya zorlayrak bu işi yaparken Random Forest'ta feature_importance_ ye göre yapar. Buradan tam anlaşılamadı ama kolayca uygulanabiliyor.

In [46]:
#LASSO
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

6 selected features


D:\YuklenenProgramlar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [47]:
embeded_lr_feature

['concave points_mean',
 'radius_worst',
 'texture_worst',
 'smoothness_worst',
 'concave points_worst',
 'symmetry_worst']

In [50]:
#Tree
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(X, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

9 selected features


In [51]:
embeded_rf_feature

['radius_mean',
 'perimeter_mean',
 'area_mean',
 'concavity_mean',
 'concave points_mean',
 'radius_worst',
 'perimeter_worst',
 'area_worst',
 'concave points_worst']

**!!!** Burada feature_importance_ özelliği olan herhangi bir estimator(tahminciyi) kullanabilirdik. LightGBM ve XGBosst bile. Aha da aşağıda sana örnek.

# Bonus

Her zaman maliyet açısından vs. mümkün olmasada tüm yöntemleri kullanarakta seçim yapılabilir.

In [52]:
feature_name = X.columns
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,radius_worst,True,True,True,True,True,5
2,concave points_worst,True,True,True,True,True,5
3,concave points_mean,True,True,True,True,True,5
4,texture_worst,True,True,True,True,False,4
5,symmetry_worst,True,True,True,True,False,4
6,smoothness_worst,True,True,True,True,False,4
7,radius_mean,True,True,True,False,True,4
8,perimeter_worst,True,True,True,False,True,4
9,perimeter_mean,True,True,True,False,True,4
10,concavity_mean,True,True,True,False,True,4


## Sonuç
Özellik mühendisliği ve özellik seçimi, herhangi bir makine öğrenim hattının kritik parçalarıdır.
Modellerimizde doğruluk için çabalıyoruz ve bu parçaları tekrar tekrar denemeden iyi bir kesinliğe ulaşılamıyor.
Burada özellik seçiminde en çok kullanılan özellik seçim tekniklerinden bazıları analtılmaya çalışılmıştır.